**Installer les dépendances et les configurations**




In [ ]:
!apt install libgl1-mesa-glx
!pip install easyocr
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import os
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np
tf.random.set_seed(12051966)
import easyocr
from sklearn.metrics import confusion_matrix , classification_report
import pandas as pd
import seaborn as sns
import pickle
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# **Entrainement**

**Supprimer les images douteuses**

parcourir un répertoire d'images,et vérifier si les fichiers sont valides en termes d'extension et de type d'image, puis supprimer les fichiers qui ne correspondent pas aux critères spécifiés.

In [ ]:
data_dir = 'images'
image_exts = ['jpeg','jpg', 'bmp', 'png']
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

**Charger les données**

charger un ensemble de données d'images à partir d'un répertoire, extrer le premier lot d'images et d'étiquettes, puis afficher les huit premières images avec leurs étiquettes correspondantes.

In [ ]:
#data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/images',labels='inferred',label_mode='binary')
data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/images')
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()
fig, ax = plt.subplots(ncols=8, figsize=(20,20))
for idx, img in enumerate(batch[0][:8]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

**Normaliser les données**

normaliser les valeurs des pixels de chaque image de l'ensemble de données en les divisant par 255, ce qui les ramène à une plage de valeurs entre 0 et 1.

In [ ]:
data = data.map(lambda x,y: (x/255, y))

**Fractionner les données**

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1

In [ ]:
print (len(data),train_size, val_size,test_size)

 diviser les données en trois ensembles distincts : l'ensemble d'entraînement, de validation et de test.

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

**Construire un modèle d'apprentissage en profondeur**

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

**Entraînement**

effectuer l'entraînement du modèle avec une planification dynamique du taux d'apprentissage

In [ ]:
logdir='log'
lr_base = 2e-4
epochs=5
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_base * 10**(epoch/epochs))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(data, epochs=epochs, validation_data=val, callbacks=[tensorboard_callback,lr_scheduler])

**Tracer les courbe**

In [ ]:
fig = plt.figure()
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
plt.plot(hist.history['val_accuracy'], color='green', label='val_accuracy')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
lrs =lr_base * (10 ** (np.arange(epochs)/epochs))
plt.figure(figsize=(10, 7))
plt.semilogx(lrs, hist.history["loss"])
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning rate vs. loss");

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

**Make prediction**

dans cette partie on fait une prediction, dans le but de tester notre modéle

préparer les données test pour ensuite faire une prediction

In [ ]:
test = data.skip(train_size+val_size).take(test_size)
test_iterator = test.as_numpy_iterator()
d=test.as_numpy_iterator().next()
X, y = d
X_norm=X/255
print(len(X_norm),len(y))
print(y)
nb_img=16

visualiser les 16 images avec leurs index et la classe à laquelle elles appartiennent

In [ ]:
fig, ax = plt.subplots(ncols=nb_img, figsize=(40,40))
for idx, img in enumerate(X[:nb_img]):
    ax[idx].imshow(img)
    legend=f'N°{idx} \n classe {y[idx]}'
    ax[idx].title.set_text(legend)

effectuer une prédiction sur les données X en utilisant le modèle model

In [ ]:
Y=model.predict(X)

effectuer une boucle sur les prédictions y_pred et afficher le type de chaque image prédite

In [ ]:
idx:int=0
y_pred =  np.argmax(Y,axis=-1)
print(y_pred)
for pred in y_pred[:nb_img]:
  if pred==1 :
    print("image ",idx," CNI")
  else:
    print("image ",idx,"autre")
  idx+=1

In [ ]:
print(y_pred[4])

In [ ]:
# test = data.skip(train_size+val_size).take(test_size)
# test_iterator = test.as_numpy_iterator()

In [ ]:
first:bool=True
for dd in test_iterator :
  xx,yy=dd
  temp=np.argmax(model.predict(xx,batch_size=32),axis=-1)

  if first :
    y_class_pred = temp
    y_class_true = np.array(yy)
    first=False
  else :
    y_class_pred=np.append(y_class_pred,temp)
    y_class_true= np.append(y_class_true,temp)
  print(f'Y_TRUE: {y_class_true}\nY_PRED= { y_class_pred}')

In [ ]:
y_class_pred,y_class_true

**Draw confusion matrix**

In [ ]:
# Source code credit for this function: https://gist.github.com/shaypal5/94c53d765083101efc0240d776a23823
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.

    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix.
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.

    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true=y_class_true,y_pred=y_class_pred)
cm

In [ ]:
class_names=["autre","cni"]

In [ ]:
for i in y :
  print (i, class_names[i])

In [ ]:
from operator import xor
print(xor(y_class_true,y_class_pred))

In [ ]:
print_confusion_matrix(cm,class_names=class_names)

**Save model**

In [ ]:
import pickle
# Open a file and use dump()
with open('models/cniClasses.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(class_names, file)
model.save("models/cni_classification")

# **Start inferance**

In [ ]:
def inferance(filename):
    model = keras.models.load_model('models/cni_classification')
    # model.summary()

    with open('models/cniClasses.pkl', 'rb') as file:
        class_names = pickle.load(file)

    img=cv2.imread(filename)
    resize = tf.image.resize(img, (256,256))
    predictions = model.predict(np.expand_dims(resize/255, 0))
    index = predictions.argmax()
    predicted_class_name = class_names[index]
    return predicted_class_name, predictions[0][index]

# **Detect cni areas and apply ocr**

**Detection**

In [ ]:
def smooth(img,type="mean",kernel=9) : # type : convol, mean, median, sd...

    img1 = img.copy()

    border = (kernel-1)/2

    border = int(border)

    if (len(img.shape)==3) :

        colored = img.shape[2]

    else :

        img = img.reshape(img.shape[0],img.shape[1],1)

        img1 =img1.reshape(img1.shape[0],img1.shape[1],1)

        colored = 1

    for color in range(0,colored) :

        lignemax = (img.shape[0]-border-1)

        lignemax = int(lignemax)

        for ligne in range(0,img.shape[0]) :

            for colonne in range(0,img.shape[1]) :

                lignemin = (ligne-border)

                if (lignemin<0) :lignemin=0

                lignemax = (ligne+border+1)

                if (lignemax>(img.shape[0]-1)) : lignemax = img.shape[0]-1

                colmin = (colonne-border)

                if colmin < 0 : colmin=0

                colmax = (colonne+border+1)

                if colmax>(img.shape[1]-1) : colmax = img.shape[1]-1

                extrait = img[lignemin:lignemax,colmin:colmax,(color):(color+1)]

                if type == "mean" :

                    new_value = np.mean(extrait)

                elif type == "median" :

                    new_value = np.median(extrait)

                elif type == "sd" :

                    new_value = np.std(extrait, ddof  =1)

                img1[ligne,colonne,color] =  new_value

    if (colored==1):

        img1 = img1.reshape(img.shape[0],img.shape[1])

    return(img1)

def get_limits(color):

    c = np.uint8([[color]])  # here insert the bgr values which you want to convert to hsv
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [ ]:
def detection(filename):
    """
    Détecte des zones spécifiques dans la CNI
    filename (str) : chemin d'accès au fichier image
    Cette fonction lit une image (CNI) à partir du nom de fichier donné, puis identifier les zones d'intérêt, et renvoie les zones détectées.
    """
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Flou gaussien pour réduire le bruit
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    # Détecter les contours de Canny
    canny = cv2.Canny(blur, 10, 70)
    thresh2 = cv2.adaptiveThreshold(canny, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 17, 2)
    #**FIND THE CONTOURS OF THE IMAGE
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Sélectionner le plus grand contour
    cnt = max(contours, key=cv2.contourArea)
    # Approximer le contour en un rectangle pour encadrer la CNI
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    # Dessiner le rectangle sur l'image d'entrée
    cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
    #Redressement de l’image
    center_rect = rect[0]
    size = rect[1]
    print(size)
    angle=rect[2]
    if rect[1][0]<rect[1][1]:
            angle = 90 -rect[2]
    else:
            angle = rect[2]
    M = cv2.getRotationMatrix2D(center_rect, angle, 1.0)
    (h1, w1) = img.shape[:2]
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((w1 * cos) + (h1 * sin))
    new_h = int((h1 * cos) + (w1 * sin))
    M[0, 2] += (new_w / 2) - center_rect[0]
    M[1, 2] += (new_h / 2) - center_rect[1]
    new_img = cv2.warpAffine(img, M, (new_w, new_h))
    #**CUT OFF OUR CNI
    gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
    # Flou gaussien pour réduire le bruit
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    canny = cv2.Canny(blur, 10, 70)
    thresh = cv2.adaptiveThreshold(canny, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 17, 2)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt2 = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(cnt2)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(new_img, [box], 0, (0, 0, 255), 2)
    x, y, w, h = cv2.boundingRect(cnt2)
    xmin, xmax = x, x+w
    ymin, ymax = y, y+h
    region = new_img[ymin:ymax, xmin:xmax]
    #resize the image
    height =738
    weight=1040
    Blue=[255,50,10]
    resized_image = cv2.resize(region, (weight, height))
    img_median = smooth(resized_image,"median",9)
    lower_blue,upper_blue=get_limits(color=Blue)
    plt.imshow(resized_image)
    plt.show()
    #**FIND THE BLUE ZONE ON CNI
    hsv = cv2.cvtColor(img_median, cv2.COLOR_BGR2HSV)
    # mask pour couleur bleu
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    #eliminer le bruit et améliorer les contours de la zone bleue
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    # trouver les contours dans le masque
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(resized_image, [box], 0, (0, 255, 0), 2)
    x, y, w, h = cv2.boundingRect(cnt)
    roi2 = resized_image[y:y+h, x:x+w]
  # Vérifier si la zone bleue se trouve en haut de l'image
    if y < resized_image.shape[0] / 2:
        print("La zone bleue est en haut de l'image.")

    else:
        print("La zone est en bas de l'image .")
        resized_image=cv2.rotate(img, cv2.ROTATE_180)
        hsv = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_blue, upper_blue)
        kernel = np.ones((5,5),np.uint8)
        mask = cv2.erode(mask,kernel,iterations = 1)
        mask = cv2.dilate(mask,kernel,iterations = 1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnt = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        x, y, w, h = cv2.boundingRect(cnt)
        roi2 = resized_image[y:y+h, x:x+w]
    #Vérifier si l'image est en recto ou verso
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    recto = False
    # Détecter les visages dans l'image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) > 0:
        recto = True
        f = faces[0]
    result=[recto,resized_image, y, h, x, w]
    return result

**Detect the front of the cni**

In [ ]:
def detectFront(image , y ,h,x,w ):
        """
        cette fonction detecte les zones du devant d'une cni
        """
        #ID
        y2 = y + 60
        roi_ID = image[y2:y2+h, x:x+w]
        #zone INFO+Photo
        y3=y2+roi_ID.shape[0]
        roi_INFO_TOTAL = image[y3:y3+(6*h), x:x+w]
        #zone MRZ
        y4 = y3 + roi_INFO_TOTAL.shape[0] #roi_INFO_TOTAL.shape[0] = heuteur de roi_info_total
        roi_MRZ = image[y4:, x:x+w]
        #zone INFO
        y5 = y2 + roi_ID.shape[0]
        x2=x+(w-680)
        roi_INFO = image[y3:y3+(4*60), x2:]
        #zone Photo
        x3=x-(w+750)
        roi_PHOTO = image[y3:y3+(6*h), x3:x3+w]
        #zone nom
        y5 = y2 + h
        x2=x+(w-680)
        roi_NOM = image[y5:y5+115, x2:]
        #zone prenom
        y6 = y5 + h
        x2=x+(w-680)
        roi_PRENOM = image[y6:y6+h, x2:]
        #zone date_naiss
        y7 = y6 + 75
        x2=x+(w-680)
        roi_DATE = image[y7:y7+85, x2:]
        #zone adresse
        y8 = y7 + 62
        x2=x+(w-700)
        roi_ADR =image[y8:y8+h, x2:]
        half_height = roi_ADR.shape[0] // 2
        roi_ADRESS = roi_ADR[:half_height, :]
        zones=[roi_PHOTO,roi_ID, roi_INFO,roi_MRZ ]
        return zones

**Detect teh back of the cni**

In [ ]:
def detectBack (image):
        """
        cette fonction detecte les zones du verso d'une cni
        """
        h, w, _ = image.shape
        half_h = h // 2
        moitie_centrale = image[half_h - (h // 4):half_h + (h // 4), :]
        return moitie_centrale

**Start ocr**

In [ ]:
def recognize(filename):
    """
    filename (str) : chemin d'accès au fichier image
    cette fonction applique l'ocr sur les zones détectées avec la fonction detection(filename) et renvoi du text.
    """
    concatenated_text = ""
    result=detection(filename)
    if result[0] == True:
        zones=detectFront(result[1],result[2],result[3],result[4],result[5])
        plt.imshow(zones[0])
        plt.show()
        for i in range(1, 4):
                image_zone = zones[i]
                image=cv2.cvtColor(image_zone,cv2.COLOR_BGR2GRAY)
                gray = cv2.cvtColor(image_zone, cv2.COLOR_BGR2GRAY)
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
                morph = cv2.morphologyEx(image_zone, cv2.MORPH_CLOSE, kernel)
                bifilter=cv2.bilateralFilter(morph, 9,19,75)
                plt.imshow(bifilter,cmap='gray')
                plt.show()
                if i== 4:
                    text=pytesseract.image_to_string(bifilter)
                    concatenated_text += text + "\n"
                else:
                    reader = easyocr.Reader(['fr'],gpu=True)
                    result = reader.readtext(bifilter,detail=1)
                    for detect in result:
                        text = detect[1]
                        concatenated_text += text + "\n"
    else:
        print("la cni est en verso ")
        info=detectBack(result[1])
        bfilter = cv2.bilateralFilter(info, 9, 9, 51)
        plt.imshow(bfilter)
        plt.show()
        reader = easyocr.Reader(['fr'],gpu=True)
        result = reader.readtext(bfilter)
        for detect in result:
            text = detect[1]
            concatenated_text += text + "\n"
    print(concatenated_text)

## **Tests**

In [ ]:
image="image_tests/page_0.jpeg"
classe , proba =inferance(image)
print(classe, str(proba) + " %")

In [ ]:

recognize(image)